In [1]:
# import pandas as pd
# import numpy as np
# from datetime import date #, datetime, timedelta

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, VotingRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor


In [2]:
from functions.functions import load_data, to_split_and_scale, experiment_initialization, predicting_by_experiment

In [3]:
price_df4, cat_features, num_features, target = load_data()

In [4]:
X_train_scaled, X_test_scaled, y_train, y_test, st_scaler, labels_dict = to_split_and_scale(price_df4, cat_features, num_features, target)


/Users/user/Documents/GitHub/poland-apartment-prices/functions/functions.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = le.fit_transform(X[col])


In [5]:
all_regressors = {
#     'linreg': LinearRegression,
#     'ridge': Ridge,
#     'lasso': Lasso,
#     'knn': KNeighborsRegressor,
#     'tree': DecisionTreeRegressor,
#     'gbr': GradientBoostingRegressor,
    'xgb': XGBRegressor
}

In [6]:
grids = {
    'linreg': {},
    'ridge': {
        "alpha": [0.0001], # list(np.logspace(-4, -1, 4))
        "solver": ["lsqr"] # ["sag", "lsqr"]
    },
    'lasso': {
        "alpha": [0.0001], # list(np.logspace(-4, 2, 7))
        "max_iter": [1000]
    },
    'knn': {
        "n_neighbors": [10], # [8, 9, 10]
        "leaf_size": [30], # [25, 30, 35]
        "p": [2],
        "algorithm": ['auto']
    },
    'tree': {
        "min_samples_split": [15], # [10, 15, 20, 25]
        "max_depth": [8], # [6, 8]
    #     "criterion": ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        "splitter": ['best'] # , 'random'
    },
    'gbr': {
        "learning_rate": [0.2], # [0.2, 0.1, 0.05]
        "max_depth": [5, 6, 7] # [5, 6, 7, 8]
    },
    'xgb': {
        "max_depth": [8], # [6, 7, 8, 9]
        "n_estimators": [200, 300],
        "learning_rate": [0.2] # [0.2, 0.1, 0.05]
    }
}

In [7]:
experiment_name = "poland_apartments"
experiment = experiment_initialization(experiment_name)

In [8]:
# Start experiments tracking UI

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

[2023-06-05 00:46:35 +0400] [73597] [INFO] Starting gunicorn 20.1.0
[2023-06-05 00:46:35 +0400] [73597] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-06-05 00:46:35 +0400] [73597] [ERROR] Retrying in 1 second.
[2023-06-05 00:46:36 +0400] [73597] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-06-05 00:46:36 +0400] [73597] [ERROR] Retrying in 1 second.
[2023-06-05 00:46:37 +0400] [73597] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-06-05 00:46:37 +0400] [73597] [ERROR] Retrying in 1 second.
[2023-06-05 00:46:38 +0400] [73597] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-06-05 00:46:38 +0400] [73597] [ERROR] Retrying in 1 second.
[2023-06-05 00:46:39 +0400] [73597] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2023-06-05 00:46:39 +0400] [73597] [ERROR] Retrying in 1 second.
[2023-06-05 00:46:40 +0400] [73597] [ERROR] Can't connect to ('127.0.0.1', 5000)
Running the mlflow server failed. Please see the logs above for details.


In [ ]:
# KIND OF INPUT DATA

X_check = pd.DataFrame({
    'city': city,
    'district': district,
    'floor': floor, 
    'rooms': rooms, 
    'sq': sq, 
    'year': year,
    'radius': radius
}, index=[0])

X_check = input_to_df(city='Warszawa', district='Śródmieście', radius=2, floor=3, rooms=2, sq=40, year=2000)


In [9]:
predicting_by_experiment(experiment, all_regressors, grids, X_train_scaled, y_train, X_test_scaled, y_test, cat_features, st_scaler, labels_dict)


/opt/homebrew/Cellar/jupyterlab/3.4.8_1/libexec/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 870, in _list_run_infos
    run_info = self._get_run_info_from_dir(r_dir)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 681, in _get_run_info_from_dir
    meta = FileStore._read_yaml(run_dir, FileStore.META_DATA_FILE_NAME)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/mlflow/store/tracking/file_store.py", line 1096, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/ho

With a probability of 87.4%, the prise will be about 632,603 PLN 


(632603.0, 0.8740818334569938)

In [462]:
# best_model = training_loop(experiment, model_class, grids[reg], X_train_scaled, y_train, X_test_scaled, y_test)['Run 1']['model']
# predict_by_input(X_check, cat_features, st_scaler, labels_dict, best_model)


With a probability of 87.4%, the prise will be about 631,622 PLN 
